https://www.npmjs.com/package/nepali-news-scraper?activeTab=code

In [4]:
# आवश्यक लाइब्रेरीहरू import गर्ने
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# eKantipur को Base URL
base_url = 'https://ekantipur.com'

# अनुरोध पठाउँदा आफूलाई ब्राउजरको रूपमा चिनाउन User-Agent हेडरको प्रयोग
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def scrape_article_detail(url):
    """
    यो function ले एउटा समाचारको लिङ्क (URL) बाट विस्तृत जानकारी निकाल्छ।
    
    Args:
        url (str): समाचारको पूरा लिङ्क।
        
    Returns:
        dict: समाचारको प्रकाशित मिति र विस्तृत विवरण भएको dictionary।
              यदि कुनै error आयो भने None return गर्छ।
    """
    if not url:
        return None
    print(f"विवरण स्क्र्याप गरिँदैछ: {url}")
    try:
        # URL मा HTTP GET request पठाउने (timeout र headers सहित)
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        
        # प्रकाशित मिति निकाल्ने
        published_date_tag = soup.select_one('.published-at')
        published_date = published_date_tag.get_text(strip=True).replace('प्रकाशित :', '').strip() if published_date_tag else 'N/A'
        
        # समाचारको पूरा विवरण (Content) निकाल्ने
        content_paragraphs = []
        # '.desc-content' भित्रका सबै <p> ट्यागहरू खोज्ने
        paragraphs = soup.select('.desc-content p')
        for p in paragraphs:
            # <script> ट्याग नभएको र अनावश्यक 'gaEvent' text नभएको अनुच्छेद मात्र छान्ने
            if not p.find('script'):
                text = p.get_text(strip=True)
                if text and 'gaEvent' not in text:
                    content_paragraphs.append(text)
        
        content = '\n'.join(content_paragraphs)
        
        return {
            'published_date': published_date,
            'content': content
        }

    except requests.exceptions.RequestException as e:
        print(f"Error: समाचारको विवरण स्क्र्याप गर्न सकिएन ({url}) - {e}")
        return None

def scrape_kantipur(limit=5):
    """
    यो function ले eKantipur को समाचार पृष्ठबाट समाचारहरू स्क्र्याप गर्छ।
    
    Args:
        limit (int): कतिवटा समाचार स्क्र्याप गर्ने भन्ने संख्या।
        
    Returns:
        list: समाचारहरूको विवरण भएको list of dictionaries।
    """
    news_url = f"{base_url}/news"
    print(f"eKantipur बाट {limit} वटा समाचार स्क्र्याप गरिँदैछ ({news_url})...")
    try:
        response = requests.get(news_url, headers=headers, timeout=20)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        
        news_items = []
        
        # '.normal' class भएका सबै समाचारहरू select गर्ने
        articles = soup.select('.normal')
        
        for article in articles[:limit]:
            title_tag = article.select_one('.teaser h2 a')
            summary_tag = article.select_one('.teaser p')
            author_tag = article.select_one('.author a')

            if title_tag:
                title = title_tag.get_text(strip=True)
                link = title_tag.get('href')
                full_link = f"{base_url}{link}" if link and link.startswith('/') else link
                summary = summary_tag.get_text(strip=True) if summary_tag else ''
                author = author_tag.get_text(strip=True) if author_tag else 'N/A'
                
                news_items.append({
                    'title': title,
                    'link': full_link,
                    'summary': summary,
                    'author': author
                })

        # प्रत्येक समाचारको विस्तृत विवरण निकाल्ने
        for item in news_items:
            # Server मा धेरै चाप नदिन हरेक request को बिचमा १ सेकेन्ड पर्खने
            time.sleep(1)
            
            details = scrape_article_detail(item['link'])
            if details:
                # निकालेको विवरणलाई update गर्ने
                item.update(details)
                    
        return news_items

    except requests.exceptions.RequestException as e:
        print(f"Error: eKantipur स्क्र्याप गर्न सकिएन - {e}")
        return []

# --- Jupyter Notebook मा चलाउने मुख्य भाग ---

# ५ वटा समाचार स्क्र्याप गर्ने
scraped_data = scrape_kantipur(limit=5)

if scraped_data:
    # स्क्र्याप गरिएको डाटालाई Pandas DataFrame मा बदल्ने
    df = pd.DataFrame(scraped_data)
    
    print("\nस्क्र्यापिङ सम्पन्न भयो!")
    print(f"जम्मा {len(df)} वटा समाचार सफलतापूर्वक स्क्र्याप गरियो।")
    
    # Jupyter Notebook मा DataFrame देखाउने
    # DataFrame को स्तम्भहरू (columns) लाई व्यवस्थित गर्ने
    display_columns = ['title', 'author', 'published_date', 'summary', 'link', 'content']
    # DataFrame मा सबै स्तम्भहरू छन् कि छैनन् भनी सुनिश्चित गर्ने
    final_df = df.reindex(columns=display_columns)
    
    display(final_df[['title', 'author', 'published_date', 'summary']])
else:
    print("\nकुनै पनि समाचार स्क्र्याप गर्न सकिएन।")

eKantipur बाट 5 वटा समाचार स्क्र्याप गरिँदैछ (https://ekantipur.com/news)...
विवरण स्क्र्याप गरिँदैछ: https://ekantipur.com/news/2025/09/12/202509121757688678.html
विवरण स्क्र्याप गरिँदैछ: https://ekantipur.com/news/2025/09/12/202509121757679700.html
विवरण स्क्र्याप गरिँदैछ: https://ekantipur.com/news/2025/09/12/202509121757676495.html
विवरण स्क्र्याप गरिँदैछ: https://ekantipur.com/news/2025/09/12/202509121757675141.html
विवरण स्क्र्याप गरिँदैछ: https://ekantipur.com/news/2025/09/12/202509121757670754.html

स्क्र्यापिङ सम्पन्न भयो!
जम्मा 5 वटा समाचार सफलतापूर्वक स्क्र्याप गरियो।


,title,author,published_date,summary
0,संविधानको धारा ६१ (४) अनुसार सुशीला कार्की अन्...,N/A,"भाद्र २७, २०८२ २०:३६",राष्ट्रपति रामचन्द्र पौडेलले सुशीला कार्कीलाई ...
1,"प्रहरीका २८ थान हतियार लुटिए, खोजी गर्न ‘विशेष...",मातृका दाहाल,"भाद्र २७, २०८२ १८:१०",जेन–जीले अगुवाई गरेको आन्दोलन पछि भड्किएको प्र...
2,कान्तिपुर टेलिभिजनको नियमित प्रसारण सुरु,N/A,"भाद्र २७, २०८२ १७:१३",कान्तिपुर टेलिभिजनको आजदेखि नियमित प्रसारण सुर...
3,राजनीतिक वार्ता र संकटकालको तयारी सँगसँगै,N/A,"भाद्र २७, २०८२ १६:५०",जेन–जी आन्दोलनको मागअनुसार अन्तरिम सरकार बनाउन...
4,कानुनी शासनको जगमा निकास निकाल्न मानव अधिकार आ...,N/A,"भाद्र २७, २०८२ १५:३७",काठमाडौं– राष्ट्रिय मानव अधिकार आयोगले नयाँ सर...
